In [ ]:
!pip install pyngrok
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install flask

In [ ]:
from flask import Flask, request, abort
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():#for colab
  ngrok.set_auth_token(userdata.get('auth_token'))
  ngrok.connect(5000)
  run_with_ngrok(app)
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    ImageMessage,
    VideoMessage,
    LocationMessage,
    )
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('access_token'))
handler = WebhookHandler(userdata.get('Channel_secret'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):

  with ApiClient(configuration) as api_client:
        action = event.message.text
        if action == "sticker":
          #回復貼圖

          reply = StickerMessage(package_id='446', sticker_id='1989')
        elif action == "image":
          #回復圖片
          reply = ImageMessage(original_content_url='http://n.sinaimg.cn/sinakd20230419ac/376/w1760h2616/20230419/7405-806c4c52700da7d864e12474610f54c7.jpg'
          ,preview_image_url='http://n.sinaimg.cn/sinakd20230419ac/376/w1760h2616/20230419/7405-806c4c52700da7d864e12474610f54c7.jpg')
        elif action =='video':
          reply = VideoMessage(original_content_url='https://www.youtube.com/watch?v=dQw4w9WgXcQ&ab_channel=RickAstley')

        #elif action =='location':
          #reply = LocationMessage(
    #title="",
    #address="地址",
    #latitude=緯度,
    #longitude=經度)

        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)

        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )
ngrok_start()#for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6ee2-35-229-217-105.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:04:50] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:04:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:05:14] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:05:29] "POST /callback HTTP/1.1" 200 -
